<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/2_maps_shift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import numpy as np

# Задаем матрицы как константы для предыдущего и текущего сканирования
MAP_PREV = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
])

MAP_CURR = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
])

def find_shift(map1: np.ndarray, map2: np.ndarray) -> tuple[int, int]:
    """
    Вычисляет смещение между двумя картами, используя фазовую корреляцию (через преобразование Фурье).

    Аргументы:
        map1: Первая карта (например, предыдущий скан).
        map2: Вторая карта (например, текущий скан).

    Возвращает:
        Кортеж (dy, dx), представляющий вертикальное и горизонтальное смещение.
    """
    # Вычисляем 2D-преобразование Фурье для обеих карт
    fft1 = np.fft.fft2(map1)
    fft2 = np.fft.fft2(map2)

    # Вычисляем спектр взаимной мощности (cross-power spectrum)
    # Это эквивалентно кросс-корреляции в пространственной области
    # и имеет преимущество в том, что он нормализован.
    # Добавляем небольшое эпсилон для избежания деления на ноль.
    cross_power_spectrum = (fft1 * np.conj(fft2)) / (np.abs(fft1) * np.abs(fft2) + 1e-10)

    # Выполняем обратное преобразование Фурье, чтобы получить матрицу фазовой корреляции
    phase_corr = np.fft.ifft2(cross_power_spectrum)

    # Смещаем нулевую частотную компоненту в центр для более легкой интерпретации
    phase_corr_shifted = np.fft.fftshift(np.abs(phase_corr))

    # Находим пик в матрице фазовой корреляции
    max_pos = np.unravel_index(np.argmax(phase_corr_shifted), phase_corr_shifted.shape)

    # Вычисляем смещение относительно центра матрицы
    center_y, center_x = np.array(phase_corr_shifted.shape) // 2
    dy = max_pos[0] - center_y
    dx = max_pos[1] - center_x

    return dy, dx

# Вычисляем смещение
dy, dx = find_shift(MAP_PREV, MAP_CURR)

# Выводим результаты
print("Предыдущая матрица (MAP_PREV):")
print(MAP_PREV)
print("\nТекущая матрица (MAP_CURR):")
print(MAP_CURR)
print(f"\nПредсказанное смещение: (dy={dy}, dx={dx})")

Предыдущая матрица (MAP_PREV):
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

Текущая матрица (MAP_CURR):
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0

In [47]:
import numpy as np
from scipy.ndimage import rotate
import numpy.fft as fft

# --- Задаем матрицы как константы ---
MAP_PREV = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
])

MAP_CURR = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
])

def find_translation(map_ref: np.ndarray, map_target: np.ndarray) -> tuple[int, int]:
    """
    Вычисляет смещение между двумя картами, используя фазовую корреляцию.
    Эта функция предназначена для нахождения линейного смещения.

    Аргументы:
        map_ref: Первая карта (опорная).
        map_target: Вторая карта (целевая).

    Возвращает:
        Кортеж (dy, dx), представляющий вертикальное и горизонтальное смещение.
    """
    fft_ref = fft.fft2(map_ref)
    fft_target = fft.fft2(map_target)

    # Вычисляем спектр взаимной мощности (cross-power spectrum)
    # Добавляем небольшое эпсилон для избежания деления на ноль.
    cross_power_spectrum = (fft_ref * np.conj(fft_target)) / (np.abs(fft_ref) * np.abs(fft_target) + 1e-10)

    # Выполняем обратное преобразование Фурье
    phase_corr_matrix = fft.ifft2(cross_power_spectrum)
    phase_corr_shifted = fft.fftshift(np.abs(phase_corr_matrix))

    # Находим пик в матрице фазовой корреляции
    max_pos = np.unravel_index(np.argmax(phase_corr_shifted), phase_corr_shifted.shape)

    # Вычисляем смещение относительно центра матрицы
    center_y, center_x = np.array(phase_corr_shifted.shape) // 2
    dy = max_pos[0] - center_y
    dx = max_pos[1] - center_x

    return dy, dx

def find_rotation_and_translation(map_ref: np.ndarray, map_target: np.ndarray) -> tuple[int, int, float]:
    """
    Оценивает поворот и смещение между двумя картами, используя итеративный поиск угла
    и фазовую корреляцию.

    Аргументы:
        map_ref: Опорная карта (например, предыдущий скан).
        map_target: Целевая карта (например, текущий скан).

    Возвращает:
        Кортеж (dy, dx, estimated_angle), представляющий вертикальное смещение,
        горизонтальное смещение и угол поворота.
    """

    max_correlation_value = -1.0 # Инициализируем максимальное значение корреляции
    best_angle = 0.0             # Инициализируем лучший угол

    # Предварительно вычисляем FFT для опорной карты, так как она не меняется в цикле.
    fft_ref_precomputed = fft.fft2(map_ref)

    # Диапазон поиска углов. Уменьшен шаг для потенциально лучшей точности.
    angle_range = np.arange(-45, 45.1, 0.5) # От -45 до +45 градусов с шагом 0.5 градуса

    print("Поиск угла поворота...")
    # Перебираем возможные углы поворота
    for angle in angle_range:
        # Поворачиваем map_target на текущий угол.
        # reshape=False сохраняет размер матрицы.
        # mode='constant', cval=0 заполняет новые пиксели нулями.
        # order=3 для бикубической интерполяции, которая дает лучшее качество.
        rotated_map_target = rotate(map_target, angle=angle, reshape=False, mode='constant', cval=0, order=3)
        # Округляем значения и приводим к целому типу (0 или 1), так как интерполяция может дать дробные.
        rotated_map_target_binary = np.round(rotated_map_target).astype(int)

        # Вычисляем фазовую корреляцию между map_ref и повернутой map_target
        fft_rotated_map_target = fft.fft2(rotated_map_target_binary)

        # Спектр взаимной мощности для определения корреляции
        cross_power_spectrum = (fft_ref_precomputed * np.conj(fft_rotated_map_target)) / \
                               (np.abs(fft_ref_precomputed) * np.abs(fft_rotated_map_target) + 1e-10)
        phase_corr_matrix = fft.ifft2(cross_power_spectrum)

        # Находим максимальное значение корреляции для текущего угла
        current_max_corr = np.max(np.abs(phase_corr_matrix))

        # Если текущая корреляция лучше, обновляем лучший угол
        if current_max_corr > max_correlation_value:
            max_correlation_value = current_max_corr
            best_angle = angle

    print(f"Найденный оптимальный угол (грубая оценка): {best_angle:.1f} градусов с максимальной корреляцией {max_correlation_value:.4f}")

    # --- Шаг 2: Коррекция поворота и оценка смещения ---
    # Поворачиваем map_target на найденный оптимальный угол, чтобы выровнять по вращению
    map_target_corrected_rotation = rotate(map_target, angle=best_angle, reshape=False, mode='constant', cval=0, order=3)
    map_target_corrected_rotation_binary = np.round(map_target_corrected_rotation).astype(int)

    # Теперь, когда поворот скорректирован, находим линейное смещение
    dy, dx = find_translation(map_ref, map_target_corrected_rotation_binary)

    return dy, dx, best_angle

# --- Выполняем оценку ---
dy, dx, estimated_angle = find_rotation_and_translation(MAP_PREV, MAP_CURR)

# --- Выводим результаты ---
print("\n" + "="*50)
print("Результаты анализа смещения и поворота:")
print("="*50)
print("Предыдущая матрица (MAP_PREV):")
print(MAP_PREV)
print("\nТекущая матрица (MAP_CURR):")
print(MAP_CURR)
print(f"\nПредсказанное смещение: (dy={dy}, dx={dx})")
print(f"Предсказанный угол поворота: {estimated_angle:.1f} градусов")

print("\n" + "="*50)
print("Примечание:")
print("Предсказанный угол определяется путем итеративного поиска и зависит от")
print("выбранного диапазона и шага углов. Точность может быть улучшена")
print("уменьшением шага или использованием более сложных методов (например,")
print("преобразования Фурье-Меллина).")
print("Ошибки интерполяции при повороте могут повлиять на точность смещения.")
print("="*50)

Поиск угла поворота...
Найденный оптимальный угол (грубая оценка): -45.0 градусов с максимальной корреляцией 0.5378

Результаты анализа смещения и поворота:
Предыдущая матрица (MAP_PREV):
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

Текущая матрица (MAP_CURR):
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0